In [1]:
import sys
import os
import kagglehub

In [2]:
!git clone https://github.com/maximus3343/movie-rec

fatal: destination path 'movie-rec' already exists and is not an empty directory.


In [3]:
cloned_repo_path = '/content/movie-rec'

if cloned_repo_path not in sys.path:
    sys.path.append(cloned_repo_path)
print(f"Added {cloned_repo_path} to sys.path")

Added /content/movie-rec to sys.path


In [4]:
import pandas as pd

from MovieEnv import MovieEnv
from PPO import PPO
from Buffer import Experience

In [5]:
path = kagglehub.dataset_download("vodclickstream/netflix-audience-behaviour-uk-movies")
dataset_name = os.listdir(path)[0]
path = os.path.join(path, dataset_name)

In [6]:
def main():
    df = pd.read_csv(path)
    N = 4  # Number of last movies watched kept as state.
    envi = MovieEnv(df, N=N, alpha=1, beta=0)

    lr = 1e-4  # Same learning rate for all modules.
    gamma = 0.95  # Discount factor.
    in_features = 128
    hidden_size = 256
    embedding_dim = 64
    id_list = df['movie_id'].unique()  # List of movies.
    K_epochs = 5
    eps_clip = 0.2
    top_k = 0

    a = PPO(in_features=in_features, out_features=envi.action_space.n, hidden_size=hidden_size, embedding_dim=embedding_dim, lr_actor=lr,
            lr_critic=lr, lr_sr=lr, gamma=gamma, N=N, K_epochs=K_epochs, eps_clip=eps_clip, movie_id_list=id_list, top_k=top_k,
            algo='monte_carlo')

    episodes = 1000
    rewards = []

    for k in range(episodes):
        state, info = envi.reset()
        done = False

        total_reward = 0

        while not done:
            action, state_value, action_logprob = a.select_action(state)
            next_state, reward, done, _, _ = envi.step(action)

            # exp = Experience(state, action, reward, next_state, done, state_value, action_logprob)
            a.add_experience(Experience(state, action, reward,
                            next_state, done, state_value, action_logprob))

            total_reward += reward

            state = next_state

        print(reward)
        a.learn()
        rewards.append(reward)

if __name__ == "__main__":
    main()



0.10976916576382588
-0.12014795726110802
0.04611212401960068
-0.2125898166590207
-0.0709290505361922
0.2486573271395874
-0.2072576096393321
-0.20881693879160554
-0.15055582847737622
-0.060127017494917154
-0.20589094708971564
-0.20760949084110786
-0.1693994691033966
-0.0032159284171516906
-0.19228360362513813
0.02307770695028772
0.19099741503419257
0.021790106858891763
-0.2126279265364693
0.04294836098867613
-0.15829912041031527
-0.004536199752458043
0.09662102566674431
-0.07298698391841864
-0.08168099335914042
-0.18266403539247777
-0.1264426927522244
-0.05662554017836019
-0.016443738884278438
-0.02725542413295661
-0.018328421491563184
-0.2126279265364693
-0.15091349673461638
-0.2010869853157735
-0.2126279265364693
-0.1421553528799895
0.21312292102723035
-0.1195472032039644
0.06571073923277826
0.11117052999046871
-0.21263427818271075
-0.07536281719502913
-0.20682040465637969
-0.2126279265364693
-0.07935549226833259
-0.2126279265364693
-0.03747492978252956
-0.18461716661172048
-0.2126183